In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
from folium.features import GeoJsonTooltip

In [2]:
#Read the geoJSON file using geopandas
geojson = gpd.read_file(r'../input/hate-crimes-data/Police Precincts.geojson')
geojson=geojson[['precinct','geometry']]

#Read the NYPD Hate Crime data using pandas
hc_df = pd.read_csv(r'../input/hate-crimes-data/NYPD_Hate_Crimes.csv')

In [3]:
hc_df.info()

In [4]:
geojson.info()

In [5]:
#Change 'Complaint Precinct Code' datatype to match 'precinct'
hc_df['Complaint Precinct Code'] = hc_df['Complaint Precinct Code'].map(str)

#Join the geojson file with hc_df on precinct code
df_final = geojson.merge(hc_df, left_on="precinct", right_on="Complaint Precinct Code", how="outer") 
df_final.head()

In [6]:
#We only want to look at Asian targeted hate crimes, so we'll create a new dataframe specific to Anti-Asian biases
aa_df = df_final[df_final['Bias Motive Description'] == 'ANTI-ASIAN']
aa_df

In [7]:
#create new df that has aggregate count of anti-Asian hate crimes per precinct
#note: value_counts() will return a series, adding .reset_index().values after value_counts will turn index into values
aggregate_df = pd.DataFrame(aa_df.precinct.value_counts().reset_index().values, columns=["precinct", "aggregate crime"])
aggregate_df

In [8]:
#we want the coordinates of the precincts so we merge our table with aa_df, drop columns we don't need and remove duplicate rows
#note: dataframe must be type geopandas.geodataframe.GeoDataFrame to be read properly later
final_df = aa_df.merge(aggregate_df, how='left')
final_df = final_df[['precinct', 'aggregate crime', 'geometry']]
final_df.drop_duplicates(subset=None, keep="first", inplace=True, ignore_index=False)
final_df 

In [9]:
#Generate map of nyc 
map = folium.Map(location=[40.693943, -73.985880], zoom_start=12)
map

In [10]:
#Create the choropleth map add it to the base map
custom_scale = (final_df['aggregate crime'].quantile((0,0.1,0.75,0.9,0.98,1))).tolist()

folium.Choropleth(
            geo_data = '../input/hate-crimes-data/Police Precincts.geojson',
            data = final_df,
            columns = ['precinct', 'aggregate crime'],  #tell folium to get the 'precinct' and 'aggregate crime' of incidents metric for each area
            #note 'precinct' must be obj datatype to work
            key_on = 'feature.properties.precinct', #grab the shared key 'precinct' from the geojson
            threshold_scale=custom_scale,
            fill_color = 'YlOrRd',
            nan_fill_color = "White", #use white color if there is no data available for the precinct
            fill_opacity = 0.7,
            line_opacity = 0.2,
            legend_name = 'Anti-Asian Hate Crimes by Precinct',
            highlight = True,
            line_color = 'black').add_to(map) 

map

In [11]:
#Add Customized Tooltips to the map
folium.features.GeoJson(
                    data = final_df,
                    name = 'Anti-Asian Hate Crimes by Precinct',
                    smooth_factor = 2,
                    style_function = lambda x: {'color':'black','fillColor':'transparent','weight':0},
                    tooltip = folium.features.GeoJsonTooltip(
                        fields = ['precinct',
                                'aggregate crime'
                               ],
                        aliases = ["Precinct:",
                                 "Aggregate Crime:"
                                ], 
                        localize = True,
                        sticky = False,
                        labels = True,
                        style = """
                            background-color: #F0EFEF;
                            border: 0px solid black;
                            border-radius: 0px;
                            box-shadow: 0px;
                        """,
                        ),
                            highlight_function = lambda x: {'weight':1,'fillColor':'grey'},
                        ).add_to(map)   

map